## Segmenting and Clustering Neighborhoods in Toronto

IBM DATA SCIENCE PROFESSIONAL CERTIFICATE - # 9 Applied Data Science Capstone - WEEK 3 | COURSERA

### 1. Data Scraping and Data Cleaning

In [201]:
# import the libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [202]:
# scrap data from the wiki page and transform the data to a data frame
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

In [203]:
# the dataframe consist of three columns: PostalCode, Borough, and Neighborhood
df[0:5]

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [204]:
# drop the rows that have "not assigned" neighbourhood
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df = df.reset_index()

In [205]:
# drop the old index column
df = df.drop(['index'], axis = 1)

In [206]:
# For the cell that has a borough but a "not assigned" neighbourhood,
# we make the neighbourhood the same as the borough
df[df.Neighbourhood == 'Not assigned']
df.at[df[df.Neighbourhood == 'Not assigned'].index, 'Neighbourhood']= "Queen's Park"
df[0:7]

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


In [207]:
# combine the neighbourhoods that have the same postal code into one row
# separate the neighbourhoods with commas
df = pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)).reset_index()
df[0:5]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [208]:
# print the number of rows of your dataframe
df.shape

(103, 3)

### 2. Add Geographical Coordinates into the Dataset

In [209]:
# I tried to use geocoder to generate the lat and lng of the coords
# But i failed to get the geographical coordinates of the neighborhoods using the Geocoder package
## !pip install geocoder
## import geocoder
## lat_lng_coords = None
##while(lat_lng_coords is None):
##  g = geocoder.google('{}, Toronto, Ontario')#.format(postal_code)
##  lat_lng_coords = g.latlng
## latitude = lat_lng_coords[0]
## longitude = lat_lng_coords[1]

In [210]:
# So I use the csv file provided by the course that has the geo coords of each postal code
# uploaded the file into the current folder and read it in
geo = pd.read_csv("Geospatial_Coordinates.csv")

In [211]:
# check the geo data
print(geo[0:7])
print("the shape of the geo data is", geo.shape)
print("the shape of our dataset is also", df.shape) 

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
5         M1J  43.744734 -79.239476
6         M1K  43.727929 -79.262029
the shape of the geo data is (103, 3)
the shape of our dataset is also (103, 3)


In [212]:
# merge the geo data and our dataframe
df = df.merge(geo, how = "inner", left_on = "Postcode", right_on = "Postal Code")

In [213]:
# check the merged dataset
df[0:3]

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",M1B,43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",M1C,43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",M1E,43.763573,-79.188711


In [214]:
# drop the extra postal code column
df = df.drop("Postal Code", axis = 1)

In [215]:
# check the merged dataset
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848


### 3. Explore and cluster the neighborhoods in Toronto

In [216]:
# import the libraries that we may need

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#### 3.1 Use geopy library to get the latitude and longitude values of Toronto

In [217]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### 3.2 Create a map of Toronto with neighborhoods superimposed on top

In [218]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [219]:
# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='cadetblue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [220]:
# display the map of toronto
map_toronto

#### 3.3 Define Foursquare Credentials and Version

In [221]:
CLIENT_ID = 'TTLEAE2OSKHU540HSUGRBWFJOF11SNV5FUVTDUQLYMKF1DZA' # your Foursquare ID
CLIENT_SECRET = '0TCLYFNQ01KYBIXQULKX1M0INDBVUJRXQPIJ0XY4BSSPKJ1N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TTLEAE2OSKHU540HSUGRBWFJOF11SNV5FUVTDUQLYMKF1DZA
CLIENT_SECRET:0TCLYFNQ01KYBIXQULKX1M0INDBVUJRXQPIJ0XY4BSSPKJ1N


#### 3.4 explore the first neighborhood

In [222]:
# Get the neighborhood's name.
df.loc[0, 'Neighbourhood']

['Rouge', 'Malvern']

In [223]:
# Get the neighborhood's latitude and longitude values
n_lat = df.loc[0, 'Latitude'] # neighborhood latitude value
n_lng = df.loc[0, 'Longitude'] # neighborhood longitude value

n_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(n_name, 
                                                               n_lat, 
                                                               n_lng))

Latitude and longitude values of ['Rouge', 'Malvern'] are 43.806686299999996, -79.19435340000001.


##### Now we want get the top 100 venues that are in this neighbourhood within a radius of 500 meters

In [224]:
# create the GET request URL
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n_lat, 
    n_lng, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=TTLEAE2OSKHU540HSUGRBWFJOF11SNV5FUVTDUQLYMKF1DZA&client_secret=0TCLYFNQ01KYBIXQULKX1M0INDBVUJRXQPIJ0XY4BSSPKJ1N&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [225]:
# Send the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c4bc35e1ed2193b468ef873'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [226]:
# borrow the get_category_type function from the Foursquare lab
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [227]:
# clean the json and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [228]:
# number of venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


#### 3.5 Explore neighbourhoods in Toronto

In [229]:
# create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [230]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues

toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

['Rouge', 'Malvern']
['Highland Creek', 'Rouge Hill', 'Port Union']
['Guildwood', 'Morningside', 'West Hill']
['Woburn']
['Cedarbrae']
['Scarborough Village']
['East Birchmount Park', 'Ionview', 'Kennedy Park']
['Clairlea', 'Golden Mile', 'Oakridge']
['Cliffcrest', 'Cliffside', 'Scarborough Village West']
['Birch Cliff', 'Cliffside West']
['Dorset Park', 'Scarborough Town Centre', 'Wexford Heights']
['Maryvale', 'Wexford']
['Agincourt']
['Clarks Corners', 'Sullivan', "Tam O'Shanter"]
['Agincourt North', "L'Amoreaux East", 'Milliken', 'Steeles East']
["L'Amoreaux West", 'Steeles West']
['Upper Rouge']
['Hillcrest Village']
['Fairview', 'Henry Farm', 'Oriole']
['Bayview Village']
['Silver Hills', 'York Mills']
['Newtonbrook', 'Willowdale']
['Willowdale South']
['York Mills West']
['Willowdale West']
['Parkwoods']
['Don Mills North']
['Flemingdon Park', 'Don Mills South']
['Bathurst Manor', 'Downsview North', 'Wilson Heights']
['Northwood Park', 'York University']
['CFB Toronto', 'Downsvi

In [231]:
# check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2243, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"[Rouge, Malvern]",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [232]:
# the number of venues that were returned for each neighborhood
toronto_venues['Neighborhood'] = toronto_venues['Neighborhood'].astype(str) # import to convert it to str! 
toronto_venues_count = toronto_venues.groupby('Neighborhood').count()
print(toronto_venues_count.shape)
toronto_venues_count.head()

(101, 6)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"[""L'Amoreaux West"", 'Steeles West']",16,16,16,16,16,16
"[""Queen's Park""]",42,42,42,42,42,42
"['Adelaide', 'King', 'Richmond']",100,100,100,100,100,100
"['Agincourt North', ""L'Amoreaux East"", 'Milliken', 'Steeles East']",2,2,2,2,2,2
['Agincourt'],4,4,4,4,4,4


In [233]:
# number of unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


#### 3.6 Analyze each neighbourhood

In [234]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

print(toronto_onehot.shape)

(2243, 279)


In [235]:
# check where is the neighborhood column in the dataframe
toronto_onehot.columns.get_loc('Neighborhood')

195

In [236]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[195]] + list(toronto_onehot.columns[:195]) + list(toronto_onehot.columns[196:])
toronto_onehot = toronto_onehot[fixed_columns]

In [237]:
print(toronto_onehot.shape)
toronto_onehot.head()

(2243, 279)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"['Rouge', 'Malvern']",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"['Highland Creek', 'Rouge Hill', 'Port Union']",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"['Highland Creek', 'Rouge Hill', 'Port Union']",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"['Guildwood', 'Morningside', 'West Hill']",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"['Guildwood', 'Morningside', 'West Hill']",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### **Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [238]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [239]:
# confirm the size
toronto_grouped.shape

(101, 279)

In [240]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

toronto_grouped['Neighborhood'] = toronto_grouped['Neighborhood'].apply(str)

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----["L'Amoreaux West", 'Steeles West']----
                  venue  freq
0    Chinese Restaurant  0.12
1  Fast Food Restaurant  0.12
2           Pizza Place  0.06
3     Indian Restaurant  0.06
4        Cosmetics Shop  0.06


----["Queen's Park"]----
                 venue  freq
0          Coffee Shop  0.24
1     Sushi Restaurant  0.05
2                Diner  0.05
3  Japanese Restaurant  0.05
4                  Gym  0.05


----['Adelaide', 'King', 'Richmond']----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----['Agincourt North', "L'Amoreaux East", 'Milliken', 'Steeles East']----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2    Accessories Store   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----['Agincourt']----
               venue  freq
0     Sandwich Place  0.25
1             Lounge  0.25
2 

##### **Now Let's put that into a *pandas* dataframe**

In [241]:
# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [242]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"[""L'Amoreaux West"", 'Steeles West']",Chinese Restaurant,Fast Food Restaurant,Cosmetics Shop,Pizza Place,Sandwich Place,Indian Restaurant,Noodle House,Japanese Restaurant,Breakfast Spot,Pharmacy
1,"[""Queen's Park""]",Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Bubble Tea Shop
2,"['Adelaide', 'King', 'Richmond']",Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Bar,Restaurant,Bakery,Clothing Store,Hotel
3,"['Agincourt North', ""L'Amoreaux East"", 'Millik...",Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,['Agincourt'],Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [243]:
# confirm the size
print(neighborhoods_venues_sorted.shape)
print(toronto_grouped.shape)

(101, 11)
(101, 279)


#### 3.7 Cluster Neighborhoods

##### **Run k-means to cluster the neighborhood into 5 clusters**

In [244]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [245]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# make the wording consistent: change neighbourhood to neighborhood, otherwise they cannot be successfully merged
df.columns = ['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
toronto_merged = df
toronto_merged['Neighborhood'] = toronto_merged['Neighborhood'].apply(str) # import to convert to str before merging!

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# drop the rows where the cluster labels are NaN so that it can be visuallized later
toronto_merged = toronto_merged.dropna(how='any')

# convert the cluster labels from float to integer so that it can be visualized later
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

# check the final dataframe
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"['Rouge', 'Malvern']",43.806686,-79.194353,0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
1,M1C,Scarborough,"['Highland Creek', 'Rouge Hill', 'Port Union']",43.784535,-79.160497,0,History Museum,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"['Guildwood', 'Morningside', 'West Hill']",43.763573,-79.188711,0,Medical Center,Rental Car Location,Spa,Electronics Store,Pizza Place,Breakfast Spot,Mexican Restaurant,Doner Restaurant,Diner,Discount Store
3,M1G,Scarborough,['Woburn'],43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,M1H,Scarborough,['Cedarbrae'],43.773136,-79.239476,0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Lounge,Creperie,Fast Food Restaurant


##### **Now we can finally visualize the resulting clusters!**

In [246]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# display the map
map_clusters

#### 3.8 Final examination of the resulting clusters

In [247]:
# cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 
                     0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
1,Scarborough,0,History Museum,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,Scarborough,0,Medical Center,Rental Car Location,Spa,Electronics Store,Pizza Place,Breakfast Spot,Mexican Restaurant,Doner Restaurant,Diner,Discount Store
3,Scarborough,0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,Scarborough,0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Lounge,Creperie,Fast Food Restaurant
6,Scarborough,0,Discount Store,Department Store,Coffee Shop,Convenience Store,Train Station,Dumpling Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,0,Bus Line,Bakery,Soccer Field,Business Service,Park,Metro Station,Bus Station,Intersection,Fast Food Restaurant,Creperie
8,Scarborough,0,Movie Theater,Motel,American Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Scarborough,0,College Stadium,Skating Rink,Café,General Entertainment,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
10,Scarborough,0,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,Comic Shop,Concert Hall,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [248]:
# cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 
                     1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1,Convenience Store,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
14,Scarborough,1,Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
48,Central Toronto,1,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [249]:
# cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 
                     2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,2,Cafeteria,Park,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
23,North York,2,Park,Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
25,North York,2,Fast Food Restaurant,Park,Food & Drink Shop,Bus Stop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
30,North York,2,Bus Stop,Park,Airport,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,East York,2,Convenience Store,Coffee Shop,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,2,Park,Bus Line,Swim School,Dim Sum Restaurant,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
50,Downtown Toronto,2,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
64,Central Toronto,2,Park,Trail,Sushi Restaurant,Jewelry Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
72,North York,2,Park,Japanese Restaurant,Bakery,Pub,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
74,York,2,Park,Fast Food Restaurant,Women's Store,Pharmacy,Market,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant


In [250]:
# cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 
                     3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,3,Baseball Field,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Dim Sum Restaurant
97,North York,3,Baseball Field,Paper / Office Supplies Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio


In [251]:
# cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 
                     4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,North York,4,Gym,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
